In [4]:

import tensorflow_datasets as tfds
import tensorflow as tf

import numpy as np
import os
import io
import pydicom
import nibabel 
#tfds.load('ct_lymph_nodes')

In [ ]:
tfds.load('ct_lymph_nodes',download_and_prepare_kwargs = {'download_config' :
    tfds.download.DownloadConfig(manual_dir = None)})

In [2]:
!pwd

/Users/Jingjing/project 1/datasets/tensorflow_datasets


In [23]:
filepath = '/Users/Jingjing/tensorflow_datasets/downloads/manual/'
patients = tf.io.gfile.listdir(os.path.join(filepath,'MED_ABD_LYMPH_MASKS'))
for patient_id in patients:
        i = 0
        if patient_id.startswith('.'):
            pass
        else:
            mask_path = os.path.join(filepath,'MED_ABD_LYMPH_MASKS',patient_id,patient_id+'_mask.nii.gz')
            with tf.io.gfile.GFile(mask_path) as f:
                mask_file = tfds.core.lazy_imports.nibabel.load(f.name).get_fdata().astype('int16')
            images = tf.io.gfile.listdir(os.path.join(filepath,'MED_ABD_LYMPH_IMAGES',patient_id))
            for file in images:
                file_name= os.path.join(filepath,'MED_ABD_LYMPH_IMAGES',patient_id,file)
                if file_name.endswith('dcm'):
                    with tf.io.gfile.GFile(file_name) as f:
                        image_file = tfds.core.lazy_imports.pydicom.read_file(f.name)
                    key = patient_id+'_'+str(i+1)

                        print( key,
                            {
                                    'image':image_file.pixel_array,
                                    'mask' : mask[:,:,i],
                                    'age' : image_file.PatientAge,
                                    'sex' :image_file.PatientSex,
                                    'body_part': image_file.BodyPartExamined

                            })
                        i+=1
        

IndentationError: unexpected indent (<ipython-input-23-13d67240b567>, line 19)

In [20]:
f.name

'/Users/Jingjing/tensorflow_datasets/downloads/manual/MED_ABD_LYMPH_MASKS/ABD_LYMPH_002/ABD_LYMPH_002_mask.nii.gz'